
# Import Python libraries.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.tree import DecisionTreeClassifier

# from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor , CatBoostClassifier

In [16]:
def get_state_dict():
    # This encoding is done based on per-capita-income-rating

    state_dict = { 33: "Andaman and Nicobar Islands",17 : "Andhra Pradesh",18 : "Arunachal Pradesh",28 : "Assam",
    32 : "Bihar",4 : "Chandigarh",25 : "Chhattisgarh",3 : "Delhi",1 : "Goa",10 : "Gujarat",5 : "Haryana",
    14 : "Himachal Pradesh",23 : "Jammu and Kashmir",30 : "Jharkhand",6 : "Karnataka",8 : "Kerala",26 : "Madhya Pradesh",
    12 : "Maharashtra",29 : "Manipur",27 : "Meghalaya",15 : "Mizoram",19 : "Nagaland",22 : "Odisha",7 : "Puducherry",
    16 : "Punjab",21 : "Rajasthan",2 : "Sikkim",13 : "Tamil Nadu",9 : "Telangana",20 : "Tripura",31 : "Uttar Pradesh",
    11 : "Uttarakhand",24 : "West Bengal" }
    
    state_dict = dict([(value, int(key)) for key, value in state_dict.items()])
    return state_dict


def get_cities_dict():
    # Ranking goes top to Bottom

    cities_list_ranked = ["Bengaluru","Pune","Ahmedabad","Chennai","Surat","Navi Mumbai","Coimbatore","Vadodara","Indore",
          "Greater Mumbai","Thane","Kalyan Dombivali","New Delhi","Noida","Ludhiana","Visakhapatnam","Pimpri Chinchwad",
    "Solapur","Raipur","Bhopal","Rajkot","Jodhpur","Madurai","Jaipur","Hyderabad","Nagpur","Lucknow","Varanasi",
    "Kanpur","Chandigarh","Ghaziabad","Gwalior","Prayagraj","Patna","Aurangabad","Agra","Meerut","Hubli Dharwad",
    "Nashik","Vasai Virar","Faridabad","Vijayawada","Ranchi","Jabalpur","Kota","Amritsar","Guwahati","Barielly",
    "Dhanbad","Srinagar" ]

    # The cities which are not present will be added at last, and one feature would be if they are present or not

    bool_city = list(np.ones((len(cities_list_ranked)),dtype = np.int64))
    
    for i in full_data.city.values:
        i = i.split("[")[0]
        if i not in cities_list_ranked:
            bool_city.append(0)
            cities_list_ranked.append(i)

    # Create a dictionary for Mapping

    cities_dict = {}
    for i,city in enumerate(cities_list_ranked):
        cities_dict[city] = i

    return cities_dict


In [17]:
def get_profess_dict(method):
    
    if method == 1 :
        # Based of Avg Salary
        
        profess_mean = []

        for i in full_data.profession.unique():
            inc = full_data[full_data.profession == i]["income"].mean()
            profess_mean.append([i,inc])

        profess_mean = np.array(profess_mean)[np.array(profess_mean)[:,1].argsort()]

        profess_dict = {}

        for i,prof in enumerate(profess_mean):
            profess_dict[prof[0]] = i
        
        return profess_dict
    
    elif method == 2 :
        # Based on risk flag

        profess_mean = []

        for i in full_data.profession.unique():
            inc = full_data[full_data.profession == i]["risk_flag"].sum()
            profess_mean.append([i,inc])

        profess_mean = np.array(profess_mean)[np.array(profess_mean)[:,1].argsort()]

        profess_dict = {}

        for i,prof in enumerate(profess_mean):
            profess_dict[prof[0]] = i

        return profess_dict

# Import Datasets

In [18]:
train_data = pd.read_csv("Training Data.csv")
test_data = pd.read_csv("Test Data.csv")
full_data = pd.concat([train_data,test_data],axis=0)

# test_data = train_data[:50000].reset_index(drop=True)
# train_data = train_data[50000:].reset_index(drop=True)

In [19]:
# Process profession --- >

profess = train_data.profession.unique()
new_profess = []

for i in profess:
    i = i.replace("_", " ")
    new_profess.append(i)
    
for i, j in zip(profess,new_profess):
    train_data.profession = train_data.profession.replace(i,j)
    
profess = test_data.profession.unique()
new_profess = []

for i in profess:
    i = i.replace("_", " ")
    new_profess.append(i)
    
for i, j in zip(profess,new_profess):
    test_data.profession = test_data.profession.replace(i,j)
    
for j,i in enumerate(train_data.city.values):
    train_data.city.values[j] = i.split("[")[0]
    
for j,i in enumerate(test_data.city.values):
    test_data.city.values[j] = i.split("[")[0]

In [20]:
# Process state  --- >

state = train_data.state.unique()
new_state = []

for i in state:
    i = i.replace("_", " ")
    new_state.append(i)
    
for i, j in zip(state,new_state):
    train_data.state = train_data.state.replace(i,j)
    
state = test_data.state.unique()
new_state = []

for i in state:
    i = i.replace("_", " ")
    new_state.append(i)
    
for i, j in zip(state,new_state):
    test_data.state = test_data.state.replace(i,j)

train_data.state = train_data.state.replace("Uttar Pradesh[5]","Uttar Pradesh")
test_data.state = test_data.state.replace("Uttar Pradesh[5]","Uttar Pradesh")

# Check how train dataset looks like.

In [21]:
train_data

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical engineer,Rewa,Madhya Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil servant,Tiruchirappalli,Tamil Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,single,rented,no,Surgeon,Kolkata,West Bengal,6,11,0
251996,251997,2843572,26,10,single,rented,no,Army officer,Rewa,Madhya Pradesh,6,11,0
251997,251998,4522448,46,7,single,rented,no,Design Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,251999,6507128,45,0,single,rented,no,Graphic Designer,Pondicherry,Puducherry,0,10,0


# Convert datatype of selected fields.

In [22]:
profession_sort = 2 # 1 or 2

train_data.state = train_data.state.map(get_state_dict()).astype(np.int64)
train_data.city = train_data.city.map(get_cities_dict())
train_data.profession = train_data.profession.map(get_profess_dict(profession_sort))

test_data.state = test_data.state.map(get_state_dict())
test_data.city = test_data.city.map(get_cities_dict())
test_data.profession = test_data.profession.map(get_profess_dict(profession_sort)) 


train_data["married"]=pd.factorize(train_data.married)[0]
train_data["house_ownership"]=pd.factorize(train_data.house_ownership)[0]
train_data["car_ownership"]=pd.factorize(train_data.car_ownership)[0]

test_data["married"]=pd.factorize(test_data.married)[0]
test_data["house_ownership"]=pd.factorize(test_data.house_ownership)[0]
test_data["car_ownership"]=pd.factorize(test_data.car_ownership)[0]

# Drop the dependent variable from the train/test dataset. 

In [23]:
xtrain=train_data.drop("risk_flag",axis=1).drop("Id",axis=1)
ytrain=train_data["risk_flag"]
test_data=test_data.drop("id",axis=1)
# ytest=test_data["risk_flag"]

In [31]:
concat = pd.concat([xtrain,test_data],axis = 0)
concat

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,1303835,23,3,0,0,0,14,50,26,3,13
1,7574516,40,10,0,0,0,6,51,12,9,13
2,3991815,66,4,1,0,0,16,52,8,4,10
3,6256451,41,2,0,0,1,6,53,22,2,12
4,5768871,47,11,0,0,0,9,54,13,3,14
...,...,...,...,...,...,...,...,...,...,...,...
27995,9955481,57,13,0,0,0,61,115,17,5,10
27996,2917765,47,9,0,0,0,16,186,26,9,14
27997,8082415,24,5,0,0,0,51,239,12,4,13
27998,9474180,51,13,0,0,1,3,236,25,13,14


In [34]:
mini = concat.min(axis=0)
maxi = concat.max(axis=0)

In [38]:
concat = (concat - mini)/maxi
xtrain = concat[:len(xtrain)]
test_data = concat[len(xtrain):]

In [39]:
concat

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,0.129353,0.025316,0.15,0.0,0.0,0.0,0.200000,0.144118,0.75000,0.214286,0.214286
1,0.756425,0.240506,0.50,0.0,0.0,0.0,0.085714,0.147059,0.31250,0.642857,0.214286
2,0.398153,0.569620,0.20,1.0,0.0,0.0,0.228571,0.150000,0.18750,0.285714,0.000000
3,0.624618,0.253165,0.10,0.0,0.0,1.0,0.085714,0.152941,0.62500,0.142857,0.142857
4,0.575860,0.329114,0.55,0.0,0.0,0.0,0.128571,0.155882,0.34375,0.214286,0.285714
...,...,...,...,...,...,...,...,...,...,...,...
27995,0.994523,0.455696,0.65,0.0,0.0,0.0,0.871429,0.335294,0.46875,0.357143,0.000000
27996,0.290747,0.329114,0.45,0.0,0.0,0.0,0.228571,0.544118,0.75000,0.642857,0.285714
27997,0.807216,0.037975,0.25,0.0,0.0,0.0,0.728571,0.700000,0.31250,0.285714,0.214286
27998,0.946393,0.379747,0.65,0.0,0.0,1.0,0.042857,0.691176,0.71875,0.928571,0.285714


In [40]:
test_data

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,0.738283,0.481013,0.95,0.0,0.0,0.0,0.885714,0.264706,0.68750,0.285714,0.214286
1,0.120470,0.050633,0.25,0.0,0.0,0.0,0.685714,0.450000,0.31250,0.357143,0.000000
2,0.889109,0.367089,0.60,0.0,0.0,0.0,0.728571,0.026471,0.31250,0.642857,0.285714
3,0.193412,0.354430,0.45,1.0,0.0,1.0,0.542857,0.935294,0.31250,0.214286,0.142857
4,0.000312,0.050633,0.90,0.0,0.0,1.0,0.771429,0.347059,0.68750,0.928571,0.071429
...,...,...,...,...,...,...,...,...,...,...,...
27995,0.994523,0.455696,0.65,0.0,0.0,0.0,0.871429,0.335294,0.46875,0.357143,0.000000
27996,0.290747,0.329114,0.45,0.0,0.0,0.0,0.228571,0.544118,0.75000,0.642857,0.285714
27997,0.807216,0.037975,0.25,0.0,0.0,0.0,0.728571,0.700000,0.31250,0.285714,0.214286
27998,0.946393,0.379747,0.65,0.0,0.0,1.0,0.042857,0.691176,0.71875,0.928571,0.285714


# Drop unnamed field from train and test dataset.

In [42]:
# xtrain_Unnamed  = xtrain.pop("Unnamed: 0")
# xtest_Unnamed  = xtest.pop("Unnamed: 0")
xtrain = np.array(xtrain)
xtest  = np.array(test_data)

# xtest  = np.array(xtest)

# Train your model

In [43]:
def train(model,xtrain,ytrain,xval,yval):
    model.fit(xtrain,ytrain,eval_set=(xval, yval),verbose_eval= True,use_best_model=True)
    y_pred = model.predict(xtrain)
    auc = roc_auc_score(ytrain, y_pred)
    print('Train ROC AUC: %f' % auc)
    y_pred = model.predict(xval)
    auc = roc_auc_score(yval, y_pred)
    print('Val ROC AUC: %f' % auc)
    return model,y_pred,auc

def get_pred(model,xtest):
    return model.predict(xtest)

In [62]:
folds = 5

y_test_oof = []
y_pred_oof = []

all_preds = {}

for itr,(train_index,test_index) in enumerate(skf(folds, random_state= 0, shuffle=True).split(xtrain,ytrain)):
    print(f"Train size {len(train_index)} | Val size {len(test_index)}")
    print()
    X_train, X_test = xtrain[train_index], xtrain[test_index]
    y_train, y_test = ytrain[train_index], ytrain[test_index]
    
    model = CatBoostClassifier(iterations=3000, depth=10, learning_rate=0.1 ,l2_leaf_reg=3, loss_function='Logloss',verbose=False,early_stopping_rounds = 200)
    model,y_pred,auc = train(model,X_train,y_train,X_test,y_test)
    
    print(f"Doing Prediction for auc {auc}")
    all_preds[str(auc) + str(itr)] = get_pred(model,test_data)
    
    y_test_oof.extend(y_test)
    y_pred_oof.extend(y_pred)
    
    print()
    
oof = roc_auc_score(y_test_oof, y_pred_oof)
print('OOF ROC AUC: %f' % oof)

Train size 201600 | Val size 50400

0:	learn: 0.6233479	test: 0.6234089	best: 0.6234089 (0)	total: 20.7ms	remaining: 1m 2s
1:	learn: 0.5671791	test: 0.5673379	best: 0.5673379 (1)	total: 57.2ms	remaining: 1m 25s
2:	learn: 0.5202679	test: 0.5208070	best: 0.5208070 (2)	total: 95.4ms	remaining: 1m 35s
3:	learn: 0.4829579	test: 0.4835719	best: 0.4835719 (3)	total: 133ms	remaining: 1m 39s
4:	learn: 0.4560598	test: 0.4568490	best: 0.4568490 (4)	total: 170ms	remaining: 1m 41s
5:	learn: 0.4338531	test: 0.4347736	best: 0.4347736 (5)	total: 206ms	remaining: 1m 43s
6:	learn: 0.4157926	test: 0.4168204	best: 0.4168204 (6)	total: 247ms	remaining: 1m 45s
7:	learn: 0.4004403	test: 0.4016366	best: 0.4016366 (7)	total: 289ms	remaining: 1m 47s
8:	learn: 0.3880797	test: 0.3896743	best: 0.3896743 (8)	total: 325ms	remaining: 1m 48s
9:	learn: 0.3786143	test: 0.3804692	best: 0.3804692 (9)	total: 366ms	remaining: 1m 49s
10:	learn: 0.3707578	test: 0.3727696	best: 0.3727696 (10)	total: 407ms	remaining: 1m 50s
11:

95:	learn: 0.2474217	test: 0.2635100	best: 0.2635100 (95)	total: 3.96s	remaining: 1m 59s
96:	learn: 0.2466671	test: 0.2629200	best: 0.2629200 (96)	total: 4.03s	remaining: 2m
97:	learn: 0.2463092	test: 0.2627083	best: 0.2627083 (97)	total: 4.09s	remaining: 2m 1s
98:	learn: 0.2456347	test: 0.2622166	best: 0.2622166 (98)	total: 4.13s	remaining: 2m
99:	learn: 0.2450490	test: 0.2617091	best: 0.2617091 (99)	total: 4.17s	remaining: 2m
100:	learn: 0.2441382	test: 0.2609541	best: 0.2609541 (100)	total: 4.21s	remaining: 2m
101:	learn: 0.2433812	test: 0.2603674	best: 0.2603674 (101)	total: 4.25s	remaining: 2m
102:	learn: 0.2429931	test: 0.2600153	best: 0.2600153 (102)	total: 4.29s	remaining: 2m
103:	learn: 0.2426141	test: 0.2597021	best: 0.2597021 (103)	total: 4.33s	remaining: 2m
104:	learn: 0.2420532	test: 0.2592815	best: 0.2592815 (104)	total: 4.36s	remaining: 2m
105:	learn: 0.2413502	test: 0.2587499	best: 0.2587499 (105)	total: 4.42s	remaining: 2m
106:	learn: 0.2409566	test: 0.2584836	best: 0.

188:	learn: 0.2116496	test: 0.2374909	best: 0.2374909 (188)	total: 8.23s	remaining: 2m 2s
189:	learn: 0.2114355	test: 0.2373381	best: 0.2373381 (189)	total: 8.27s	remaining: 2m 2s
190:	learn: 0.2113285	test: 0.2372858	best: 0.2372858 (190)	total: 8.3s	remaining: 2m 2s
191:	learn: 0.2110007	test: 0.2370216	best: 0.2370216 (191)	total: 8.34s	remaining: 2m 1s
192:	learn: 0.2108313	test: 0.2369263	best: 0.2369263 (192)	total: 8.38s	remaining: 2m 1s
193:	learn: 0.2107001	test: 0.2368496	best: 0.2368496 (193)	total: 8.41s	remaining: 2m 1s
194:	learn: 0.2104256	test: 0.2366644	best: 0.2366644 (194)	total: 8.45s	remaining: 2m 1s
195:	learn: 0.2102056	test: 0.2365109	best: 0.2365109 (195)	total: 8.49s	remaining: 2m 1s
196:	learn: 0.2100093	test: 0.2364384	best: 0.2364384 (196)	total: 8.53s	remaining: 2m 1s
197:	learn: 0.2097584	test: 0.2362883	best: 0.2362883 (197)	total: 8.57s	remaining: 2m 1s
198:	learn: 0.2097120	test: 0.2362760	best: 0.2362760 (198)	total: 8.6s	remaining: 2m 1s
199:	learn: 

282:	learn: 0.1943508	test: 0.2270962	best: 0.2270962 (282)	total: 11.8s	remaining: 1m 53s
283:	learn: 0.1941557	test: 0.2270140	best: 0.2270140 (283)	total: 11.9s	remaining: 1m 53s
284:	learn: 0.1939804	test: 0.2269274	best: 0.2269274 (284)	total: 11.9s	remaining: 1m 53s
285:	learn: 0.1938909	test: 0.2269170	best: 0.2269170 (285)	total: 11.9s	remaining: 1m 53s
286:	learn: 0.1937260	test: 0.2268412	best: 0.2268412 (286)	total: 12s	remaining: 1m 53s
287:	learn: 0.1935648	test: 0.2267345	best: 0.2267345 (287)	total: 12s	remaining: 1m 53s
288:	learn: 0.1934532	test: 0.2267069	best: 0.2267069 (288)	total: 12.1s	remaining: 1m 53s
289:	learn: 0.1932304	test: 0.2265836	best: 0.2265836 (289)	total: 12.1s	remaining: 1m 53s
290:	learn: 0.1930835	test: 0.2265100	best: 0.2265100 (290)	total: 12.2s	remaining: 1m 53s
291:	learn: 0.1929978	test: 0.2264360	best: 0.2264360 (291)	total: 12.2s	remaining: 1m 53s
292:	learn: 0.1928590	test: 0.2263898	best: 0.2263898 (292)	total: 12.3s	remaining: 1m 53s
293

377:	learn: 0.1828132	test: 0.2216691	best: 0.2216691 (377)	total: 15.4s	remaining: 1m 47s
378:	learn: 0.1825825	test: 0.2215937	best: 0.2215937 (378)	total: 15.5s	remaining: 1m 47s
379:	learn: 0.1824393	test: 0.2215361	best: 0.2215361 (379)	total: 15.5s	remaining: 1m 46s
380:	learn: 0.1823808	test: 0.2215098	best: 0.2215098 (380)	total: 15.6s	remaining: 1m 46s
381:	learn: 0.1822987	test: 0.2214756	best: 0.2214756 (381)	total: 15.6s	remaining: 1m 46s
382:	learn: 0.1821532	test: 0.2214192	best: 0.2214192 (382)	total: 15.6s	remaining: 1m 46s
383:	learn: 0.1821113	test: 0.2214051	best: 0.2214051 (383)	total: 15.7s	remaining: 1m 46s
384:	learn: 0.1820580	test: 0.2213645	best: 0.2213645 (384)	total: 15.7s	remaining: 1m 46s
385:	learn: 0.1819370	test: 0.2213526	best: 0.2213526 (385)	total: 15.7s	remaining: 1m 46s
386:	learn: 0.1817831	test: 0.2212599	best: 0.2212599 (386)	total: 15.8s	remaining: 1m 46s
387:	learn: 0.1816538	test: 0.2212237	best: 0.2212237 (387)	total: 15.8s	remaining: 1m 46s

...

1011:	learn: 0.1483377	test: 0.2146605	best: 0.2146492 (1003)	total: 39s	remaining: 1m 16s
1012:	learn: 0.1483109	test: 0.2146459	best: 0.2146459 (1012)	total: 39s	remaining: 1m 16s
1013:	learn: 0.1482647	test: 0.2146253	best: 0.2146253 (1013)	total: 39.1s	remaining: 1m 16s
1014:	learn: 0.1482178	test: 0.2146139	best: 0.2146139 (1014)	total: 39.1s	remaining: 1m 16s
1015:	learn: 0.1481879	test: 0.2146001	best: 0.2146001 (1015)	total: 39.1s	remaining: 1m 16s
1016:	learn: 0.1481626	test: 0.2145961	best: 0.2145961 (1016)	total: 39.2s	remaining: 1m 16s
1017:	learn: 0.1481105	test: 0.2145981	best: 0.2145961 (1016)	total: 39.2s	remaining: 1m 16s
1018:	learn: 0.1480973	test: 0.2145981	best: 0.2145961 (1016)	total: 39.2s	remaining: 1m 16s
1019:	learn: 0.1480761	test: 0.2145859	best: 0.2145859 (1019)	total: 39.3s	remaining: 1m 16s
1020:	learn: 0.1480446	test: 0.2145829	best: 0.2145829 (1020)	total: 39.3s	remaining: 1m 16s
1021:	learn: 0.1480119	test: 0.2145958	best: 0.2145829 (1020)	total: 39.4s

1101:	learn: 0.1455039	test: 0.2143849	best: 0.2143627 (1076)	total: 42.3s	remaining: 1m 12s
1102:	learn: 0.1454956	test: 0.2143929	best: 0.2143627 (1076)	total: 42.3s	remaining: 1m 12s
1103:	learn: 0.1454664	test: 0.2144042	best: 0.2143627 (1076)	total: 42.3s	remaining: 1m 12s
1104:	learn: 0.1454389	test: 0.2144000	best: 0.2143627 (1076)	total: 42.4s	remaining: 1m 12s
1105:	learn: 0.1454145	test: 0.2144226	best: 0.2143627 (1076)	total: 42.4s	remaining: 1m 12s
1106:	learn: 0.1453954	test: 0.2144280	best: 0.2143627 (1076)	total: 42.5s	remaining: 1m 12s
1107:	learn: 0.1453616	test: 0.2144368	best: 0.2143627 (1076)	total: 42.5s	remaining: 1m 12s
1108:	learn: 0.1453443	test: 0.2144412	best: 0.2143627 (1076)	total: 42.5s	remaining: 1m 12s
1109:	learn: 0.1453023	test: 0.2144352	best: 0.2143627 (1076)	total: 42.6s	remaining: 1m 12s
1110:	learn: 0.1452963	test: 0.2144415	best: 0.2143627 (1076)	total: 42.6s	remaining: 1m 12s
1111:	learn: 0.1452525	test: 0.2144288	best: 0.2143627 (1076)	total: 4

1191:	learn: 0.1429863	test: 0.2142952	best: 0.2142827 (1189)	total: 45.6s	remaining: 1m 9s
1192:	learn: 0.1429509	test: 0.2142949	best: 0.2142827 (1189)	total: 45.6s	remaining: 1m 9s
1193:	learn: 0.1429291	test: 0.2142893	best: 0.2142827 (1189)	total: 45.6s	remaining: 1m 9s
1194:	learn: 0.1429035	test: 0.2142878	best: 0.2142827 (1189)	total: 45.7s	remaining: 1m 8s
1195:	learn: 0.1428625	test: 0.2142925	best: 0.2142827 (1189)	total: 45.7s	remaining: 1m 8s
1196:	learn: 0.1428470	test: 0.2143005	best: 0.2142827 (1189)	total: 45.7s	remaining: 1m 8s
1197:	learn: 0.1428364	test: 0.2142906	best: 0.2142827 (1189)	total: 45.8s	remaining: 1m 8s
1198:	learn: 0.1428129	test: 0.2142813	best: 0.2142813 (1198)	total: 45.8s	remaining: 1m 8s
1199:	learn: 0.1427832	test: 0.2142762	best: 0.2142762 (1199)	total: 45.8s	remaining: 1m 8s
1200:	learn: 0.1427313	test: 0.2142687	best: 0.2142687 (1200)	total: 45.9s	remaining: 1m 8s
1201:	learn: 0.1426832	test: 0.2142364	best: 0.2142364 (1201)	total: 45.9s	remai

1281:	learn: 0.1408098	test: 0.2142930	best: 0.2142317 (1207)	total: 48.8s	remaining: 1m 5s
1282:	learn: 0.1407922	test: 0.2142949	best: 0.2142317 (1207)	total: 48.8s	remaining: 1m 5s
1283:	learn: 0.1407792	test: 0.2142862	best: 0.2142317 (1207)	total: 48.9s	remaining: 1m 5s
1284:	learn: 0.1407537	test: 0.2142865	best: 0.2142317 (1207)	total: 48.9s	remaining: 1m 5s
1285:	learn: 0.1407363	test: 0.2142953	best: 0.2142317 (1207)	total: 48.9s	remaining: 1m 5s
1286:	learn: 0.1407018	test: 0.2142885	best: 0.2142317 (1207)	total: 49s	remaining: 1m 5s
1287:	learn: 0.1406768	test: 0.2142803	best: 0.2142317 (1207)	total: 49s	remaining: 1m 5s
1288:	learn: 0.1406694	test: 0.2142850	best: 0.2142317 (1207)	total: 49.1s	remaining: 1m 5s
1289:	learn: 0.1406530	test: 0.2142822	best: 0.2142317 (1207)	total: 49.1s	remaining: 1m 5s
1290:	learn: 0.1406332	test: 0.2142859	best: 0.2142317 (1207)	total: 49.1s	remaining: 1m 5s
1291:	learn: 0.1406159	test: 0.2142917	best: 0.2142317 (1207)	total: 49.2s	remaining

1374:	learn: 0.1388182	test: 0.2142720	best: 0.2142317 (1207)	total: 52.3s	remaining: 1m 1s
1375:	learn: 0.1387858	test: 0.2142619	best: 0.2142317 (1207)	total: 52.3s	remaining: 1m 1s
1376:	learn: 0.1387625	test: 0.2142455	best: 0.2142317 (1207)	total: 52.4s	remaining: 1m 1s
1377:	learn: 0.1387447	test: 0.2142653	best: 0.2142317 (1207)	total: 52.4s	remaining: 1m 1s
1378:	learn: 0.1387239	test: 0.2142756	best: 0.2142317 (1207)	total: 52.4s	remaining: 1m 1s
1379:	learn: 0.1387046	test: 0.2142731	best: 0.2142317 (1207)	total: 52.5s	remaining: 1m 1s
1380:	learn: 0.1386807	test: 0.2142671	best: 0.2142317 (1207)	total: 52.5s	remaining: 1m 1s
1381:	learn: 0.1386606	test: 0.2142750	best: 0.2142317 (1207)	total: 52.5s	remaining: 1m 1s
1382:	learn: 0.1386538	test: 0.2142707	best: 0.2142317 (1207)	total: 52.6s	remaining: 1m 1s
1383:	learn: 0.1386387	test: 0.2142765	best: 0.2142317 (1207)	total: 52.6s	remaining: 1m 1s
1384:	learn: 0.1386258	test: 0.2142854	best: 0.2142317 (1207)	total: 52.6s	remai

In [63]:
roc_auc_score(model.predict(X_test),y_test)

0.7666140232791528

In [ ]:
"""
With Classifier :
OOF ROC AUC: 0.843004 : (iterations=15, depth=3, learning_rate=0.5, loss_function='CrossEntropy',verbose=False)
OOF ROC AUC: 0.874740 : (iterations=14, depth=3, learning_rate=0.5, loss_function='Logloss',verbose=False)

With Regressor :

OOF ROC AUC: 0.890554 : (iterations=1000, depth=5, learning_rate=1.0, loss_function='RMSE',verbose=False)
OOF ROC AUC: 0.930738 : (iterations=3000, depth=10, learning_rate=0.1 ,l2_leaf_reg=3, loss_function='RMSE',verbose=False)




"""

In [ ]:
# RMSE, MultiRMSE, MAE, Quantile, LogLinQuantile, Poisson, MAPE, Lq or custom objective object

In [64]:
# get_pred(DTClassifier,test_data)

NameError: name 'DTClassifier' is not defined

In [ ]:
# cols = np.array(train_data.columns)[1:-1]
# impt = DTClassifier.feature_importances_

# df = pd.DataFrame([cols,impt]).T.rename({0:"Feature",1:"Importance"},axis=1)

In [74]:
def rank_data(sub):
    sub = pd.DataFrame(base)[0]
    sub = sub.rank() / sub.rank().max()
    return sub

In [75]:
# grid = {'learning_rate': [0.03, 0.1,1.0],
#     'depth': [4, 6, 10],
#     'l2_leaf_reg': [1, 3, 5, 7, 9]}

# model = CatBoostRegressor(iterations=2000,loss_function='RMSE',verbose=False)
# grid_search_result = model.grid_search(grid, 
#                                    X=xtrain, 
#                                    y=ytrain, 
#                                    stratified = True,
#                                    cv = 5,
#                                    plot=True)

In [65]:
sample_sub = pd.read_csv("Sample Prediction Dataset.csv")
base = np.zeros(len(test_data))
for val in all_preds.values():
    base += np.array(val)

base/= folds
sample_sub

,id,risk_flag
0,1,0
1,2,0
2,3,1
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,1
27997,27998,0
27998,27999,0


In [68]:
sample_sub = pd.read_csv("Sample Prediction Dataset.csv")

sample_sub['risk_flag'] = base (yha pr base me predictions aaye hai)

In [69]:
sample_sub.to_csv("734.csv",index=False)

In [67]:
base

array([0., 0., 0., ..., 0., 0., 0.])

In [66]:
rank_data(base)

0        0.418935
1        0.507519
2        0.118999
3        0.744356
4        0.149039
           ...   
27995    0.334959
27996    0.448225
27997    0.427615
27998    0.796721
27999    0.832351
Name: 0, Length: 28000, dtype: float64

0        0.007795
1        0.013241
2       -0.001260
3        0.164691
4       -0.000410
           ...   
27995    0.004356
27996    0.009124
27997    0.008128
27998    0.237952
27999    0.329370
Name: 0, Length: 28000, dtype: float64

In [1]:
import pandas as pd

In [2]:
sa = pd.read_csv("734.csv")

In [7]:
sa.risk_flag = 1*(sa.risk_flag.values >=0.5 )

In [9]:
sa.to_csv("predict73.csv",index=False)